In [1]:
import numpy as np
import pandas as pd
import pandas_datareader.data as web
import quandl as ql
import datetime as dt
import statsmodels.regression.linear_model as rg
import statsmodels.tools.tools as ct

In [3]:
# 3.5. Assets and Market Relationship (Systematic Risk)

# Asset/Portfolio: U.S. Large Cap Stock Market
# Market: U.S. Total Stock Market
# Risk Free Rate: U.S. Total Money Market Mean
# Fama-French-Carhart Factors: U.S. Total Stock Market Premium (Mkt-Rf), U.S. Total Stock Market Size Premium (SMB),
#                              U.S. Total Stock Market Investment Style Premium (HML), Risk Free Rate (Rf)

# Data Query and Range Delimiting
expretquery1 = web.DataReader(['SPY', 'VTI'], 'yahoo', '2006-04-01', '2016-05-01')
cexpret = expretquery1['Close']
mexpretall = cexpret.asfreq('M', method='pad')
mexpretall.columns = ['mlarge', 'mstocks']

expretquery2 = ql.get("USTREASURY/YIELD.1", collapse="monthly")
expretquery2 = expretquery2['2006-04-01':'2016-04-30']
expretquery2.columns = ['mcash']
mexpretall.loc[:, 'mcash'] = expretquery2['mcash']


In [4]:
# Modify Path to Data File
expretquery3 = pd.read_csv('C:\\Users\\nisha\\Documents\\Data Science\\Portfolio Analytics\\Expected-Excess-Returns-Data.txt', index_col='Date',
                           parse_dates=True)
mexpretall.loc[:, 'mmomentum'] = expretquery3['mmomentum']
mexpretall.loc[:, 'mspremium'] = expretquery3['mspremium']
mexpretall.loc[:, 'msize'] = expretquery3['msize']
mexpretall.loc[:, 'mstyle'] = expretquery3['mstyle']
mexpretall.loc[:, 'mrisk'] = expretquery3['mrisk']

expretquery4 = ql.get("FRED/CPIAUCSL", collapse="monthly")
expretquery4 = expretquery4['2006-04-01':'2016-04-30']
expretquery4.columns = ['mcpi']
mexpretall.loc[:, 'mcpi'] = expretquery4['mcpi']


In [5]:
# Monthly and Annual Returns Calculation
mexpretall.loc[:, 'mstocksret'] = mexpretall.loc[:, 'mstocks'] / mexpretall.loc[:, 'mstocks'].shift(1) - 1
mexpretall.loc[:, 'astocksret'] = mexpretall.loc[:, 'mstocks'] / mexpretall.loc[:, 'mstocks'].shift(12) - 1

mexpretall.loc[:, 'mlargeret'] = mexpretall.loc[:, 'mlarge'] / mexpretall.loc[:, 'mlarge'].shift(1) - 1
mexpretall.loc[:, 'alargeret'] = mexpretall.loc[:, 'mlarge'] / mexpretall.loc[:, 'mlarge'].shift(12) - 1

mexpretall.loc[:, 'mcashret'] = (mexpretall.loc[:, 'mcash'] / 100) / 12
mexpretall.loc[:, 'acashret'] = mexpretall.loc[:, 'mcash'] / 100

mexpretall.loc[:, 'mmomentumret'] = mexpretall.loc[:, 'mmomentum'] / 100
mexpretall.loc[:, 'mspremiumret'] = mexpretall.loc[:, 'mspremium'] / 100
mexpretall.loc[:, 'msizeret'] = mexpretall.loc[:, 'msize'] / 100
mexpretall.loc[:, 'mstyleret'] = mexpretall.loc[:, 'mstyle'] / 100
mexpretall.loc[:, 'mriskret'] = mexpretall.loc[:, 'mrisk'] / 100

mexpretall.loc[:, 'mcpiret'] = mexpretall.loc[:, 'mcpi'] / mexpretall.loc[:, 'mcpi'].shift(1) - 1
mexpretall.loc[:, 'acpiret'] = mexpretall.loc[:, 'mcpi'] / mexpretall.loc[:, 'mcpi'].shift(12) - 1


In [6]:
# 3.5.1. Capital Asset Pricing Model CAPM

mstocksmean = np.nanmean(mexpretall.loc[:, 'mstocksret'])
mlargemean = np.nanmean(mexpretall.loc[:, 'mlargeret'])
mcashmean = np.nanmean(mexpretall.loc[:, 'mcashret'])


C:\Users\nisha\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: Mean of empty slice
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\nisha\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: Mean of empty slice
  after removing the cwd from sys.path.


In [7]:
# CAPM Beta Coefficient
# First convert all NaN value to 0 as cov function doesn't have exception
mexpretall[np.isnan(mexpretall)] = 0
mlargecov = np.cov(mexpretall.loc[:, 'mstocksret'], mexpretall.loc[:, 'mlargeret'])
mstocksstdev2 = np.nanstd(mexpretall.loc[:, 'mstocksret']) ** 2
mbetacapm = mlargecov.item(1) / mstocksstdev2


C:\Users\nisha\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  


In [8]:
# CAPM Jensen's Alpha Intercept
malphacapm = mlargemean - (mcashmean + mbetacapm * (mstocksmean - mcashmean))


In [9]:
# CAPM Expected Monthly Returns
mexpretall.loc[:, 'mcapmret'] = mexpretall.loc[:, 'mcashret'] \
                               + mbetacapm * (mexpretall.loc[:, 'mstocksret'] - mexpretall.loc[:, 'mcashret'])


In [10]:
# CAPM Expected Annualized Monthly Returns
mexpretall.loc[:, 'mcapmretcuma'] = np.cumprod(mexpretall.loc[:, 'mcapmret'] + 1) ** (12 / len(mexpretall)) - 1
lastdate = dt.datetime(2016, 3, 31)
mcapmreta = mexpretall.at[lastdate, 'mcapmretcuma']


In [11]:
# 3.5.2. Fama-French-Carhart Factors Model

# Fama-French-Carhart Regression
# Add Constant Column as OLS regression doesn't include it
mexpretall.loc[:, 'mconstant'] = ct.add_constant(mexpretall)
# Calculate U.S. Large Cap Stocks Premium
mexpretall.loc[:, 'mlpremiumret'] = mexpretall.loc[:, 'mlargeret'] - mexpretall.loc[:, 'mriskret']
# Calculate OLS Regression
mregfactorsffc = ['mmomentumret', 'mspremiumret', 'msizeret', 'mstyleret', 'mconstant']
mregressionffc = rg.OLS(mexpretall.loc[:, 'mlpremiumret'], mexpretall.loc[:, mregfactorsffc],
                        hasconst=bool).fit()


In [12]:
# Fama-French-Carhart Beta Coefficients
mmomentumbeta = mregressionffc.params[0]
mspremiumbetaffc = mregressionffc.params[1]
msizebeta = mregressionffc.params[2]
mstylebeta = mregressionffc.params[3]


In [13]:
# Fama-French-Carhart Alpha Intercept
malphaffc = mregressionffc.params[4]


In [14]:
# Fama-French-Carhart Expected Monthly Returns
mexpretall.loc[:, 'mffcret'] = mexpretall.loc[:, 'mriskret'] + malphaffc \
                               + mspremiumbetaffc * mexpretall.loc[:, 'mspremiumret'] \
                               + msizebeta * mexpretall.loc[:, 'msizeret'] \
                               + mstylebeta * mexpretall.loc[:, 'mstyleret'] \
                               + mmomentumbeta * mexpretall.loc[:, 'mmomentumret']


In [15]:
# Fama-French-Carhart Expected Annualized Monthly Returns
mexpretall.loc[:, 'mffcretcuma'] = np.cumprod(mexpretall.loc[:, 'mffcret'] + 1) ** (12 / len(mexpretall)) - 1
mffcreta = mexpretall.at[lastdate, 'mffcretcuma']


In [16]:
# 3.5.3. Arbitrage Pricing Theory Model APT

# Arbitrage Pricing Theory Regression
mregfactorsapt = ['mspremiumret', 'mcpiret', 'mconstant']
mregressionapt = rg.OLS(mexpretall.loc[:, 'mlpremiumret'], mexpretall.loc[:, mregfactorsapt],
                        hasconst=bool).fit()


In [17]:
# Arbitrage Pricing Theory Beta Coefficients
mspremiumbetaapt = mregressionapt.params[0]
mcpibeta = mregressionapt.params[1]


In [18]:
# Arbitrage Pricing Theory Alpha Intercept
malphaapt = mregressionapt.params[2]


In [19]:
# Arbitrage Pricing Theory Expected Monthly Returns
mexpretall.loc[:, 'maptret'] = mexpretall.loc[:, 'mriskret'] + malphaapt \
                               + mspremiumbetaapt * mexpretall.loc[:, 'mspremiumret'] \
                               + mcpibeta * mexpretall.loc[:, 'mcpiret']


In [20]:
# Arbitrage Pricing Theory Expected Annualized Monthly Returns
mexpretall.loc[:, 'maptretcuma'] = np.cumprod(mexpretall.loc[:, 'maptret'] + 1) ** (12 / len(mexpretall)) - 1
maptreta = mexpretall.at[lastdate, 'maptretcuma']


In [21]:
# 3.5.4. Expected Excess Returns Comparison Tables

# Expected Returns Third Comparison
# Compounded Annual Growth Rate need to be calculated again
mexpretall.loc[:, 'mlargecuma'] = np.cumprod(mexpretall.loc[:, 'mlargeret'] + 1) ** (12 / len(mexpretall)) - 1
mlargereta = mexpretall.at[lastdate, 'mlargecuma']
# Comparison Table
aretcompdata3 = [{'0': '', '1': 'mlargereta', '2': 'mcapmreta', '3': 'mffcreta', '4': 'maptreta'},
        {'0': 'Annualized Monthly Return', '1': mlargereta, '2': mcapmreta, '3': mffcreta,'4': maptreta}]
aretcomptable3 = pd.DataFrame(aretcompdata3)
print("")
print(aretcomptable3)
print("")



                           0           1          2            3           4
0                             mlargereta  mcapmreta     mffcreta    maptreta
1  Annualized Monthly Return           0        NaN  6.04255e-05  7.6776e-05



In [22]:
# Expected Excess Returns Comparison
# Annualize CAPM, Fama-French-Carhart and Arbitratge Pricing Theory Models Alpha Intercept
malphacapma = malphacapm * np.sqrt(12)
malphaffca = malphaffc * np.sqrt(12)
malphaapta = malphaapt * np.sqrt(12)
# Comparison Table
aeretcompdata = [{'0': '', '1': 'malphacapma', '2': 'malphaffca', '3': 'malphaapta'},
        {'0': 'Annualized Monthly Excess Return', '1': malphacapma, '2': malphaffca, '3': malphaapta}]
aeretcomptable = pd.DataFrame(aeretcompdata)
print("")
print(aeretcomptable)
print("")



                                  0            1           2           3
0                                    malphacapma  malphaffca  malphaapta
1  Annualized Monthly Excess Return          NaN -0.00297044 -0.00255747



In [23]:
# Beta Coefficients Comparison
mbetacompdata = [{'0': '(Systematic Risk)', '1': 'mcapmret', '2': 'mffcret', '3': 'maptret'},
        {'0': 'Market Beta:', '1': mbetacapm, '2': mspremiumbetaffc, '3': mspremiumbetaapt},
        {'0': 'Size Beta:', '1': '', '2': msizebeta, '3': ''},
        {'0': 'Style Beta:', '1': '', '2': mstylebeta, '3': ''},
        {'0': 'Momentum Beta:', '1': '', '2': mmomentumbeta, '3': ''},
        {'0': 'Inflation Beta:', '1': '', '2': '', '3': mcpibeta}]
mbetacomptable = pd.DataFrame(mbetacompdata)
print("")
print(mbetacomptable)
print("")



                   0         1           2          3
0  (Systematic Risk)  mcapmret     mffcret    maptret
1       Market Beta:       NaN  0.00197951  0.0037047
2         Size Beta:            0.00680506           
3        Style Beta:           -0.00491146           
4     Momentum Beta:             -0.001851           
5    Inflation Beta:                       -0.0850816



In [24]:
# CAPM Residual Variance (Unsystematic Risk)
mcapmresvar = np.nanvar(rg.OLS(mexpretall.loc[:, 'mlargeret'], mexpretall.loc[:, 'mstocksret']).fit().resid)
mcapmresvara = mcapmresvar * np.sqrt(12)
print('CAPM Annualized Monthly Residual Variance (Unsystematic Risk):', mcapmresvara)
print("")

CAPM Annualized Monthly Residual Variance (Unsystematic Risk): 0.0

